 **Resume template used : https://www.overleaf.com/1362122341rnqyqdtgszvb**


---

tex file is parsed for all content measurements 

---
The resume is arranged in the form of a dictionary with each section forming the keys and the content forming the values

---
**Content measurement is to determine:**



*  Grammatical and formatting errors along with providing a formatted and 
grammatically correct content

*  Whether the skills section is relevant or the experience section is relevant (according to given JD)

*   Job roles suited for the candidate according to the skillset (based on static nlp based rules for 4 broad job roles)








---
For the sake of simplicity, I have considered the following JD of an SDE intern:


**Responsibilities**

Document and test new software applications
Assess new application ideas
Research competitor offerings
Develop applications (coding, programming)
Interview beta testers


**Requirements**

Computer Science/Network Engineering, Engineering, or Mathematics major
Excellent analytical and mathematics skills
Ability to work in teams
Experience programming in C, C++, or Java or Python,Django or AWS


**Benefits**

Practical experience with a wide variety of software engineering tasks
Collaborating hand-in-hand with skilled teams of software engineers
Shadowing, mentoring, and training opportunities with seasoned professionals
Opportunity to participate in networking events and company meetings
Flexible schedule for students
Compensation available



---
---


**References:**
 * For the sample JD: https://www.internships.com/employer/resources/internships/computer

 * For Software Development Engineer role: https://www.monster.com/career-advice/article/software-engineer-skills

 * For FullStack Developer role: https://www.simplilearn.com/skills-required-to-become-a-full-stack-developer-article

 * For Cybersecurity Developer role: https://www.simplilearn.com/tutorials/cyber-security-tutorial/cyber-security-skills

 * For DevOps Engineer role: https://www.naukri.com/kubernetes-jobs


In [1]:
!pip install texsoup
!pip install docx2txt
!pip install language_tool_python
!pip install spacy

import language_tool_python

# using the tool for Indian English for grammar check  
tool = language_tool_python.LanguageTool('en-IN') 

from TexSoup.tokens import tokenize 
from TexSoup.tokens import tokenize_string
from TexSoup.category import categorize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
m_tool = Matcher(nlp.vocab)

from collections import Counter
from spacy.matcher import PhraseMatcher

import re
import pprint
import docx2txt

  Created wheel for texsoup: filename=TexSoup-0.3.1-py3-none-any.whl size=27666 sha256=98d3e15702f4dbf468406ceabdfa7b225f71a7287a2136cfc3dfe302bf4d2904
  Stored in directory: /root/.cache/pip/wheels/2b/8e/c6/7ee3f9f9776fd18c0ccb04492902ed42225b6dfab3c68f14c0
Successfully built texsoup
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3980 sha256=18c1b7ca520fcaba350e9574ee943b006cd5abbf077fb9c0f5904fefc75f8a7c
  Stored in directory: /root/.cache/pip/wheels/b7/20/b2/473e3aea9a0c0d3e7b2f7bd81d06d0794fec12752733d1f3a8
Successfully built docx2txt


Unzipping /tmp/tmpqvhbcyif.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [2]:
#mount Google drive on colab
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from TexSoup import TexSoup
with open('/content/gdrive/MyDrive/mCSE/resume4.tex') as f: #change path as required
  soup= TexSoup(f)
#print(soup)

In [4]:
section=[]
cont=[]
sec=list(soup.find_all('rSection')) #according to format 3, it is "section" tag
length=len(sec)
cmd=soup.count("itemize")
#print(cmd)
for i in range(length):
  p=str(sec[i].args[0])
  section.append( re.sub(r"[{}]+", "", p))
print("sections: "+ str(section) )

cont=list(soup.find_all('rSection'))

sections: ['Education', 'SKILLS', 'PROJECTS', 'Extra-Curricular Activities', 'EXPERIENCE', 'Work History']


In [5]:
#print(cont[0].text)

In [5]:
#extracting section content as list of list items
token_text=[]
cleaned_text=[]
for i in range(len(cont)):
  token_text.append(tokenize_string(categorize(cont[i].text[1:])))
for j in range(len(token_text)):
  cleaned_text.append(re.sub(r"[\t\n\\]","",str(token_text[j])))

#print(*cleaned_text)
#print(token_text)

In [6]:
#putting the content of each section in a dictionary form
dict2=dict(zip(section,cleaned_text))
pprint.pprint(dict2)

{'EXPERIENCE': 'SDE Oct, 2021 - Present Mindtree Ltd. Bengaluru, Karnataka, '
               'India -3pt  Achieved 15% growth for Mindtree Stock using '
               'Technical and Soft skills. SDE Sept, 2021 - Oct, 2021 '
               'Intellisoft Technologies Bengaluru, Karnataka, India -3pt  '
               "Created client's app using React JS and received Appreciation "
               'Award.   ',
 'Education': ' B.Tech in Computer Science and Engineering, BML Munjal '
              'University August 2021GPA: 3.5Minor in Cybersecurity Member of '
              'Martial Arts Fight Club Member of HackBMU ',
 'Extra-Curricular Activities': ' Singing    Dancing.       Martial Arts',
 'PROJECTS': '-1.25emProject 1HTML, Node.js and Express.js, MySQL, CSS, React, '
             'DevOps Toolinghttps://github.com/Mrityu503/tindogGitHub -3pt  '
             'Created a dating website for dogs.      Retrieved data from '
             'database. Project 2Python, '
             'Djangohttp

In [7]:
#grammar checking function according downloaded Indian English Tool

def grammar(sectionstring):
  # given text  
  text = str(sectionstring)
   
# getting the matches  
  matches = tool.check(text)  
  
# defining some variables  
  Mistakes = []  
  Corrections = []  
  start = []  
  end = []  

# using the for-loop  
  for rules in matches:  
      if len(rules.replacements) > 0:  
          start.append(rules.offset)  
          end.append(rules.errorLength + rules.offset)  
          Mistakes.append(text[rules.offset : rules.errorLength + rules.offset])  
          Corrections.append(rules.replacements[0])  
  
# creating new object  
  NewText = list(text) 
# rewriting the correct passage  
  for n in range(len(start)):  
      for i in range(len(text)):  
        NewText[start[n]] = Corrections[n]  
        if (i > start[n] and i < end[n]):  
            NewText[i] = ""  
  
  NewText = "".join(NewText)  
  
# printing the text  
  #print(my_NewText)
  return [NewText,list(zip(Mistakes, Corrections))]


In [8]:
#appending corrected text list and a lit of (mistake, correction) per section.
catch_text=[]
for i in range(len(section)):
  catch_text.append(grammar(dict2[section[i]]))

In [9]:
#printing corrected text per section

print("Corrected text for each section are as follows :- ")
print("\n")
for j in range(len(section)):
  print(str(section[j]) + " : " + str(catch_text[j][0]))

Corrected text for each section are as follows :- 


Education :  B. Tech in Computer Science and Engineering, BML Munjal University August 2021GPA: 3.5Minor in Cybersecurity Member of Martial Arts Fight Club Member of HackBMU 
SKILLS :  @ >l @6ex l Languages & Javascript, Python, HTML, CSS, PostgreSQL, GraphQL Frameworks & React, Redux, Node.js, Express, Django, Mocha Tools & Git, Docker, TravisCI, Kubernetes, AWS, Wireshark, Kali LinuxSoft Skills & Time Management, Teamwork, Communication, Problem-Solving, Leadership, Accountability
PROJECTS : -1.25emProject 1HTML, Node.js and Express.js, MySQL, CSS, React, DevOps Toolinghttps://github.com/Mrityu503/tindogGitHub -3pt Created a dating website for dogs.  Retrieved data from database. Project 2Python, Djangohttps://github.com/Mrityu503/twitter-sentiment-trainingGitHub -3pt Created a sentiment analysis feature for ML. Project 3HTML, CSS, JShttps://github.com/Mrityu503/dice-gameGitHub -3pt Created a simple dice game for fun. 
Extra-Curric

In [10]:
#printing (mistake(s), correction(s)) per section. This includes spelling mistakes, grammar structure, space formatting errors or any such errors as defined in the language tool. 
#In case of no mistake(s) in the section content, a null list is returned.

print("Wrong , Corrected Grammar for each sectiom are as follows :-")
print("\n")
for k in range(len(section)):
  print(str(section[k]) + " : " + str(catch_text[k][1]))

Wrong , Corrected Grammar for each sectiom are as follows :-


Education : [('Tech', ' Tech')]
SKILLS : [('Problem Solving', 'Problem-Solving')]
PROJECTS : [('  ', ' '), ('     ', ' '), ('  ', ' '), ('  ', ' ')]
Extra-Curricular Activities : [('    ', ' '), ('      ', ' ')]
EXPERIENCE : [('  ', ' '), ('  ', ' '), ('  ', ' ')]
Work History : []


In [11]:
JD=docx2txt.process("/content/gdrive/MyDrive/mCSE/JD.docx")
#print(JD)

In [12]:
#docx JD is cleaned and stored as a string

JD1=re.sub(r"\s+"," ", JD, flags = re.I)
print(JD1)

Responsibilities Document and test new software applications Assess new application ideas Research competitor offerings Develop applications (coding, programming) Interview beta testers Requirements Computer Science/Network Engineering, Engineering, or Mathematics major Excellent analytical and mathematics skills Ability to work in teams Experience programming in C, C++, or Java or Python,Django or AWS Benefits Practical experience with a wide variety of software engineering tasks Collaborating hand-in-hand with skilled teams of software engineers Shadowing, mentoring, and training opportunities with seasoned professionals Opportunity to participate in networking events and company meetings Flexible schedule for students Compensation available


In [13]:
#Print the similarity scores
#Since here education section is missing in the JD, the similarity score is coming out as 0

vector = CountVectorizer()
l1=[]
cl_sk=[]
cv=[]
count_matrix=[]
scores=[]
for i in range(len(section)):
  l1.append(dict2[section[i]])
  cl_sk.append(l1[1:len(section)])
  cv=[str(cl_sk),JD1]
  count_matrix.append(vector.fit_transform(cv))
  print("\nSimilarity Scores of " + section[i] + " section : ")
  #print(cosine_similarity(count_matrix[i]))
  scores=cosine_similarity(count_matrix[i])
  print(scores)


Similarity Scores of Education section : 
[[0. 0.]
 [0. 1.]]

Similarity Scores of SKILLS section : 
[[1.         0.05340459]
 [0.05340459 1.        ]]

Similarity Scores of PROJECTS section : 
[[1.         0.07133159]
 [0.07133159 1.        ]]

Similarity Scores of Extra-Curricular Activities section : 
[[1.         0.07111953]
 [0.07111953 1.        ]]

Similarity Scores of EXPERIENCE section : 
[[1.         0.08493113]
 [0.08493113 1.        ]]

Similarity Scores of Work History section : 
[[1.         0.08999718]
 [0.08999718 1.        ]]


In [14]:
#get the match percentage

for j in range(len(section)):
  matchPercentage = cosine_similarity(count_matrix[j])[0][1] * 100
  matchPercentage = round(matchPercentage, 2) # round to two decimal
  print("Your " + section[j] + " matches about "+ str(matchPercentage)+ "% of the job description.")



Your Education matches about 0.0% of the job description.
Your SKILLS matches about 5.34% of the job description.
Your PROJECTS matches about 7.13% of the job description.
Your Extra-Curricular Activities matches about 7.11% of the job description.
Your EXPERIENCE matches about 8.49% of the job description.
Your Work History matches about 9.0% of the job description.


In [107]:
#Making static rules for 4 types of job roles

sde = [nlp(text) for text in ('Java', 'Python', 'C#','.Net','Mean','Ruby')]
fullstackdev = [nlp(text) for text in ('HTML','CSS','Javascript','Node.js','Java','Python','C#','PHP','Perl','Ruby','DBMS')]
cybersecuritydev = [nlp(text) for text in ('Kali Linux','Python','Shell Scripting','AWS','Virtual Machines','VPN','C','C++','Javascript','PHP','HTML','SQL','Go Lang','Assembly Language')]
devops = [nlp(text) for text in ('Kubernetes','Docker','Azure','Node.js')]

In [108]:
#Adding rules defined above to the matcher

matcher = PhraseMatcher(nlp.vocab)
matcher.add('Software Development Engineer', None, *sde)
matcher.add('Full Stack Developer', None, *fullstackdev)
matcher.add('Cyber Security Developer', None, *cybersecuritydev)
matcher.add('DevOps Engineer', None, *devops)


In [109]:
count=[]
match_with=nlp(dict2["SKILLS"]) #key name comes from dict2 printed above
matches = matcher(match_with)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID like "Software Development Engineer"
    span = match_with[start : end]  # get the matched slice 
    count.append((rule_id, span.text))
#print("\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(count).items()))
   

In [ ]:
#pprint.pprint(count)

In [ ]:
#print(count[12][1])
#print(len(count))

In [114]:
#extracting job role list from count list where rules were matched with the skills sections

role=[]
for i in range(len(count)):
  role.append(count[i][0])
#print(role)

#duplicate instances of job roles are removed

res = []
[res.append(x) for x in role if x not in res]
print(res)

['Full Stack Developer', 'Cyber Security Developer', 'Software Development Engineer', 'DevOps Engineer']


In [111]:
#function to display skills matched for each job role as described in the rule

def skill_list(pattern):
  Skills=[]
  for i in range(len(count)):
    if count[i][0] == pattern:
      Skills.append(count[i][1])
  return(Skills)

In [112]:
#calling the skill_list function block for each job role

catch_skill=[]
for j in range(len(res)):
  catch_skill.append(skill_list(res[j]))
#print(str(res[0]) + str(catch_skill[0]))
#print(catch_skill)
#print(res)

In [113]:
#What job role is suited for which of the skills as per matching from the rule

print("Job roles available for you : based on your skills are as listed below:- ")
print("\n")
dict3=dict(zip(res,catch_skill))
pprint.pprint(dict3)

Job roles available for you : based on your skills are as listed below:- 


{'Cyber Security Developer': ['Javascript', 'Python', 'HTML', 'AWS'],
 'DevOps Engineer': ['Node.js', 'Docker', 'Kubernetes'],
 'Full Stack Developer': ['Javascript', 'Python', 'HTML', 'CSS', 'Node.js'],
 'Software Development Engineer': ['Python']}
